In [1]:
from sim import NBodySimulation, animate3d, generateDisk3Dv3
import numpy as np
import json

In [38]:


def gen_params():
    return {
        'nbStars': int(np.random.randint(100, 101)),
        'radius': float(np.random.uniform(0.5, 2.5)),
        'Mass': float(np.random.uniform(1.0, 3.0)),
        'zOffsetMax': float(np.random.uniform(0, 0.5)),
        'gravityCst': 1.0,
        'distribution': 'hernquist',
        'offset': [float(np.random.uniform(-1, 1)), float(np.random.uniform(-1, 1)), float(np.random.uniform(-1, 1))],
        'initial_vel': [float(np.random.uniform(-0.1, 0.1)), float(np.random.uniform(-0.1, 0.1)), float(np.random.uniform(-0.1, 0.1))],
        'clockwise': int(np.random.choice([1, 0])),
        'angle': [float(np.random.uniform(-1, 1)*2*np.pi), float(np.random.uniform(-1, 1)*2*np.pi), float(np.random.uniform(-1, 1)*2*np.pi)]
    }

def generate_scene_2gals():
    params1 = gen_params()
    params2 = gen_params()
    print(params1)
    print(params2)

    particles1 = generateDisk3Dv3(**params1)
    particles2 = generateDisk3Dv3(**params2)

    t_end = 10.0
    dt = 0.01
    softening = 0.1
    G = 1.0

    particles = particles1 + particles2
    sim = NBodySimulation(particles, G, softening, dt)

    pos, vel, acc, KE, PE, _, masses, types = sim.run(t_end=t_end, save_states=True)
    print(pos.shape)
    
    # Convert all arrays to lists
    pos = np.array(pos).transpose(2, 0, 1)
    vel = np.array(vel).transpose(2, 0, 1)
    acc = np.array(acc).transpose(2, 0, 1)
    KE = KE.flatten().astype(float).tolist()  # Ensure floats
    PE = PE.flatten().astype(float).tolist()  # Ensure floats
    masses = masses.flatten().astype(float).tolist()  # Ensure floats

    print(pos.shape)
    print(vel.shape)
    print(acc.shape)

    frames = []
    for i in range(len(pos)):
        frames.append({
            'frame': int(i),  # Ensure the frame index is an int
            'pos': pos[i].tolist(),
            'vel': vel[i].tolist(),
            'acc': acc[i].tolist()
        })

    final_json = {
        'galaxy1_params': params1,
        'galaxy2_params': params2,
        'dt': float(dt),
        'softening': float(softening),
        'G': float(G),
        't_end': float(t_end),
        'masses': [float(m) for m in masses],
        'types': types,
        'KE': KE,
        'PE': PE,
        'frames': frames
    }

    return json.dumps(final_json, indent=4)


def generate_dataset(n_scenes=1, window_size=3):
    # the objective is to generate samples from n scenes, of 3 frames each, saving positions, velocities and accelerations, the idea is to predict acceleration from the first frame for the next 2 frames to integrate position and velocity
    dataset = []
    for i in range(n_scenes):
        scene = generate_scene_2gals()
        scene = json.loads(scene)
        frames = scene['frames']
        masses = scene['masses']
        for j in range(len(frames)-window_size):
            sample = {
                'scene': i,
                'frame': j,
                'masses': masses,
                'pos': frames[j]['pos'],
                'vel': frames[j]['vel'],
                'acc': frames[j]['acc'],
            }
            for k in range(1, window_size):
                sample['pos_next{}'.format(k)] = frames[j+k]['pos']
                sample['vel_next{}'.format(k)] = frames[j+k]['vel']
                sample['acc_next{}'.format(k)] = frames[j+k]['acc']
            dataset.append(sample)

    return dataset

dataset = generate_dataset(1, 3)


{'nbStars': 100, 'radius': 1.2233319325988472, 'Mass': 1.0471439663931703, 'zOffsetMax': 0.33352644401524323, 'gravityCst': 1.0, 'distribution': 'hernquist', 'offset': [-0.8914588981339868, -0.9489557350159583, -0.8080929146500702], 'initial_vel': [0.011019707113007676, 0.0848561977605157, 0.06506129658101384], 'clockwise': 0, 'angle': [-3.3895535283929252, 5.354523438365304, 0.014740515383350111]}
{'nbStars': 100, 'radius': 1.0895231168603616, 'Mass': 2.920978986105149, 'zOffsetMax': 0.35138012554165793, 'gravityCst': 1.0, 'distribution': 'hernquist', 'offset': [0.5692516584377638, 0.4508505935223568, 0.13828059620338884], 'initial_vel': [0.09406618012980336, -0.03160465188233999, 0.07435991916994705], 'clockwise': 1, 'angle': [4.618789919547556, -3.329051985509494, -0.2039424814726258]}


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:03<00:00, 285.25it/s]


(202, 3, 1001)
(1001, 202, 3)
(1001, 202, 3)
(1001, 202, 3)
